<a href="https://colab.research.google.com/github/Robsy14/Se-ales-y-sistemas/blob/main/ParcialesSyS/Parcial1_LinaJamioySyS_2024_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.


## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

## Lina Robsana Jamioy Tisoy
1006908436

# Instrucciones

-- Para recibir el crédito total, sus respuestas deben estar justificadas de manera clara, detallada y concreta, mostrando los procedimientos y razonamientos paso a paso.

-- Está permitido el uso de herramientas de inteligencia artificial (IA). Si las utiliza, por favor declare explícitamente cómo fueron empleadas en la resolución de cada pregunta. Incluya los prompts (consultas) y las iteraciones realizadas con las IA durante el desarrollo del parcial.

-- La entrega del parcial debe realizarse antes de las 23:59 del 5 de diciembre de 2024 al correo electrónico amalvarezme@unal.edu.co mediante un enlace de GitHub.

-- Los códigos deben estar debidamente comentados en las celdas correspondientes y explicados en celdas de texto (markdown). Los códigos que no incluyan comentarios ni discusiones no serán considerados en la evaluación final.

# Pregunta 1
*   Elemento de la lista
*   Elemento de la lista



Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital de 5 bits con frecuencia de muestreo de $5kHz$, entrada análoga de -3.3 a 3.3 [v], aplicado a la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$?. Realizar la simulación del proceso de digitalización incluyendo al menos 3 ciclos de la señal $x(t)$.

En caso de que la digitalización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada. El convesor debe permitir configurar la cantidad de bits, rango de la entrada análoga y la frecuencia de muestreo, indicándole al usuario si dicha frecuencia es apropiada o no, y graficar la señal continua, discreta y digital.

In [ ]:
#importar las librerias
import matplotlib.pyplot as plt #libreria para graficar
import numpy as np #libreria para trabajar con arrreglos
from scipy.spatial.distance import cdist # funcion cdist de scipy.spatial.distance

In [ ]:
#parametrod de la señal
A1 = 0.3 #amplitud 1
A2 = 0.6 #amplitud 2
A3 = 0.1 #amplitud 3

w1 = 1000*np.pi #frecuencia 1
w2 = 2000*np.pi #frecuencia 2
w3 = 11000*np.pi #frecuencia 3

u1 = -np.pi/4 #fase 1
u2 = 0 #fase 2
u3 = -np.pi #fase 3

#calculamos las frecuencias y periodos de nuestra señal
F1 = (w1/(2*np.pi)) #frecuencia 1
F2 = (w2/(2*np.pi)) #frecuencia 2
F3 = (w3/(2*np.pi)) #frecuencia 3

T1 = 1/F1 #periodo 1
T2 = 1/F2 #periodo 2
T3 = 1/F3 #periodo 3

T = (1/500)#periodo de la señal cuasiperiodica
#Fs= 5000  # frecuencia de muestreo del ejercicio
Fs = 4*max(F1,F2,F3)# frecuencia de muestreo cumpliendo nyquist
print(Fs)
Ts = 1/Fs #periodo de muestreo

#definimos el vector tiempo
ti = 0 #empezamos en 0
tf = 3*T #3 periodos hacia delante

tv = np.arange(ti,tf,Ts) #tiempo de muestreo


#construimos la señal
x1 = A1*np.cos(w1*tv+u1)
x2 = A2*np.sin(w2*tv+u2)
x3 = A3*np.cos(w3*tv+u3)
x = x1 + x2 + x3  # señanal original


#graficas de las señales
plt.figure(figsize=(12, 8))
plt.plot(tv,x,'#E9967A',label='$x(t) = 0.3\cos(1000π*t-π/4)- 0.6 \sin(2000πt)+ 0.1\cos(11000πt-π)$')
plt.plot(tv,x1,'b',label='$x_1(t) = 0.3\cos(1000πt-π/4)$')
plt.plot(tv,x2,'#2E8B57',label='$x_2(t) = 0.6\sin(2000πt)$')
plt.plot(tv,x3,'#008080',label='$x_3(t) = 0.1\cos(11000πt-π)$')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#funcion del cero y pendiente
def my_ceropen(x,ymin, ymax):
  '''
  Codigo base para simular proceso de cero y pendiente
  Se ingresa arreglo de numpy y los valores min  y max despues de cero y pendiente
  '''
  xmax = max(x) # x.max()
  xmin = min(x) # x.min()
  m = (ymax - ymin)/(xmax - xmin)
  c = ymin - m*xmin
  yv = m*x + c
  return yv

ymin = -3.3
ymax = 3.3
xcp = my_ceropen(x, ymin, ymax)


#graficas
plt.figure(figsize=(12, 6))

plt.subplot(121) # 1 = #filas, 2 = #columnas, 1 = # dibujo
plt.plot(tv,x,'#E9967A',label='$x(t)$')
plt.xlabel('tiempo [s]')
plt.ylabel('x[tk]')
plt.legend()

plt.subplot(122)

plt.plot(tv,xcp,'#8c564b',label='cero_pendiente')
plt.xlabel('tiempo [s]')
plt.ylabel('y[tk]')
plt.subplots_adjust(hspace=0.5,wspace=0.5) #ajustar espaciado entre gráficas
plt.legend()

plt.show()



In [ ]:
#función cuantización punto a punto

def my_cuantizador(yn, vq) : #yn punto a #cuantizar, vq vector de estados

  Ne = vq.shape[0] #tamaño vector de estados
  dn = cdist(yn.reshape(-1,1),vq.reshape(-1,1))#distancia yn a vector estados, reshape(-1,1) asegura vectores columna para poder utilizar cdist
  ind = np.argmin(dn) #posición distancia min
  return vq[ind]

#bits elegidos por el ususario
#5 numero de bits propuestos en el ejercicio
while True:
    try:
        nbits = int(input("Ingrese el número de bits (mayor que cero): "))
        if nbits > 0:
            break
        else:
            print("El número de bits debe ser mayor que cero.")
    except ValueError:
        print("Entrada inválida. Ingrese un número entero.")


num_est = 2**nbits #numero de estados de cuantificacion
vq = np.linspace(ymin,ymax,num_est) #vector llamado vq con los niveles de cuantificacion

#recorrer vector cero y spam discretizado para cuantizar

for i in range(xcp.shape[0]):
   if i==0: #primera muestra
     yq = my_cuantizador(xcp[i],vq)
   else:
     yq = np.append(yq,my_cuantizador(xcp[i],vq)) #apilar datos cuantizando si i>0


#graficar señales

plt.plot(tv,xcp,c='#E9967A', label='xcp')#cs continua

plt.stem(tv,xcp, label='xcp_dis')#cs discreta

#estados de cuantización

for v in vq:
  plt.plot(tv,v*np.ones(len(tv)))

# graficar señal Digital (discreta + cuantizada)

plt.plot(tv,yq,c='#8c564b',marker='o',label='ydig', linewidth=2)

plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show()

# Pregunta 2

---



Se dispone de un sistema modelado como una "caja negra" (ver celdas de código). Su tarea es analizar y comprobar mediante simulaciones si el sistema cumple con las propiedades de linealidad e invariancia en el tiempo. En caso de que el sistema sea lineal e invariante con el tiempo, determine su respuesta al impulso y utilice esta respuesta para calcular la salida del sistema ante la siguiente señal:

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
# cargar sistema
FILEID = "1J9rhh0wWHZSBd8XmWGt1ZpCsMDuoUFmm"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O P1_model.zip && rm -rf /tmp/cookies.txt
!unzip -o P1_model.zip
!dir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_
from scipy.signal import firwin, freqz, lfilter, filtfilt

#sistema pregunta 2
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs #frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo
signal_u = np.heaviside(t,1) # función heaviside
y_u = my_system.predict(signal_u)


# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

### Verificamos si cumple la linealidad y la invariancia en el tiempo para saber si el sitema es SLIT

Linealidad:  $y(t)=H \{a1x1(t)+a2x2(t)\} = a1H\{x1(t)\}+a2H\{x2(t)\}$

Invariancia con el tiempo:
$y(t) = H\{x(t)\} ; y(t-to) = H\{x(t-to)\}$


In [ ]:
#Comprobar linelidad

t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo

x1 = 2
x2 = 3

signal_u = np.heaviside(t,1) # función heaviside
y_u = my_system.predict(signal_u)

signal_v = np.sin(100 * np.pi * t) #funcion sen
y_v = my_system.predict(signal_v)

s1 = x1 * y_u  #señal 1 multiplicada por estacalar
s2 = x2 * y_v  #señal 2 multiplicada por estacalar

si = x1 * signal_u + x2 * signal_v #suma de las entradas
y_comb = my_system.predict(si)

sf = s1 + s2

# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, si, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,si, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

Como la señal de salida y de entrada son iguales se comprueba Linealidad

In [ ]:
#Comprobar si es invariancia con el tiempo
to = 0.005
signal_u = np.heaviside(t - to,1) # función heaviside
y_u = my_system.predict(signal_u)


# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

la señal es SLIT, como vemos en los resultados de las celdas anteriores la señal es lineal e invariante en el tiempo

Ahora se trabaja en la respuesta al impulso



In [ ]:
t_adjusted = t[:-1]
h_n = np.diff(y_u) * fs  #encontramos la respuesta al impulso derivando la salida del sistema
plt.figure()

# Ajustamos el tamaño de t para que coincida con h_n
t_adjusted = t[:-1]  # Eliminamos el último elemento de t

plt.plot(t_adjusted, h_n, label='Respuesta al impulso')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.legend()
plt.grid()

In [ ]:
n = np.round(t*fs) # numero de muestras

x_1= np.sin(100 * np.pi * n)
x_2= np.sin(600 * np.pi * n)

x_n = x_1 + x_2

plt.plot(n, x_n, 'g', label='Salida del sistema')
plt.stem(n, x_n, label="x(t) = sin(100πt) + sin(600πt)")
plt.title("Gráfico de la entrada del sistema")
plt.xlabel("Tiempo (t)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Realizar la convolución
y_output = np.convolve(x_n, h_n, mode='same') / fs

# Visualización de la salida
plt.figure()
plt.plot(n, y_output, 'g', label='Salida del sistema')
plt.stem(n, y_output, label='Salida del sistema')
plt.xlabel('Muestras')
plt.ylabel('Amplitud')
plt.legend()
plt.grid()
plt.title('Salida del sistema y[n]')
plt.show()